#### Introduce package

In [3]:
from ase.io.cube import read_cube_data

#### Data loading 

In [4]:
data, atoms = read_cube_data('qe_files/n2/output/homo-restricted.cube')

<_io.TextIOWrapper name='qe_files/n2/output/pspw_hf_LiH.movecs' mode='r' encoding='UTF-8'>

#### one-electron integrals

In [1]:
def calc_h_ij(self, p, c_ip_up, c_ip_dw, atoms, cell_volume):
    # Kinetic energy
    iTj_up = calc_matrix_elements.iTj(self.p, self.c_ip_up)
    iTj_dw = calc_matrix_elements.iTj(self.p, self.c_ip_dw)

    # Nuclear repulsion
    iUj_up = calc_matrix_elements.iUj(
        p,
        c_ip_up,
        atoms,
        cell_volume,
    )
    iUj_dw = calc_matrix_elements.iUj(
        p,
        c_ip_dw,
        atoms,
        cell_volume,
    )

    h_ij_up = iTj_up - iUj_up
    h_ij_dw = iTj_dw - iUj_dw

    return h_ij_up, h_ij_dw

#### Two-electron integrals

In [ ]:
def calc_eri(self):
    # Calculate ERIs via pair density
    assert (
        self.wfc_up_obj.gamma_only
    ), "Calculating ERIs via pair densities is only implemented for the gamma-point!"
    assert (
        self.wfc_dw_obj.gamma_only
    ), "Calculating ERIs via pair densities is only implemented for the gamma-point!"

    eri_up: np.ndarray = (
        eri_pair_densities.eri_gamma(p=self.p, c_ip_up=self.c_ip_up)
        / self.wfc_up_obj.cell_volume
    )
    eri_dw: np.ndarray = (
        eri_pair_densities.eri_gamma(p=self.p, c_ip_up=self.c_ip_dw)
        / self.wfc_dw_obj.cell_volume
    )
    eri_dw_up: np.ndarray = (
        eri_pair_densities.eri_gamma(
            p=self.p, c_ip_up=self.c_ip_up, c_ip_dw=self.c_ip_dw
        )
        / self.wfc_dw_obj.cell_volume
    )
    # eri_up_dw: np.ndarray = (
    #     eri_pair_densities.eri_gamma(
    #         p=self.p, c_ip_up=self.c_ip_dw, c_ip_dw=self.c_ip_up
    #     )
    #     / self.wfc_dw_obj.cell_volume
    # )
    eri_up_dw: np.ndarray = eri_dw_up.swapaxes(0, 1).swapaxes(2, 3)

    return eri_up, eri_dw, eri_dw_up, eri_up_dw